# Declarative Workflows with Report/Form

The `lionpride.work` module provides declarative workflow orchestration through the **Report/Form** pattern.

**Key Concepts:**
- **Assignment DSL**: `"inputs -> outputs"` declares data flow
- **Form**: Pure data contract - assignment parsing, input/output tracking
- **Report**: Namespace + schema registry using class attributes
- **Automatic Scheduling**: Dependencies inferred from field dataflow, independent forms run in parallel

In [ ]:
from __future__ import annotations

from dotenv import load_dotenv
from pydantic import BaseModel, Field

load_dotenv()

from lionpride import Session, iModel
from lionpride.work import Form, Report, flow_report, parse_assignment

## 1. Assignment DSL

The assignment syntax declares data flow between fields:

```
"input1, input2 -> output1, output2"
"branch_name: input -> output"  # Optional branch prefix
```

In [ ]:
# Basic assignment parsing
branch, inputs, outputs = parse_assignment("a, b -> c")
print(f"Branch: {branch}")
print(f"Inputs: {inputs}")
print(f"Outputs: {outputs}")

In [ ]:
# Assignment with branch prefix
branch, inputs, outputs = parse_assignment("orchestrator: context, plan -> result")
print(f"Branch: {branch}")
print(f"Inputs: {inputs}")
print(f"Outputs: {outputs}")

## 2. Form - Pure Data Contract

A `Form` is a unit of work defined by its assignment. No execution logic - just data flow declaration.

In [ ]:
# Create a form
form = Form(assignment="topic, context -> analysis")

print(f"Assignment: {form.assignment}")
print(f"Input fields: {form.input_fields}")
print(f"Output fields: {form.output_fields}")
print(f"Filled: {form.filled}")

In [ ]:
# Check workability (all inputs available)
available_data = {"topic": "AI adoption", "context": "enterprise"}
print(f"Is workable: {form.is_workable(available_data)}")

# Missing input
partial_data = {"topic": "AI adoption"}
print(f"Is workable (partial): {form.is_workable(partial_data)}")

In [ ]:
# Get inputs for execution
inputs = form.get_inputs(available_data)
print(f"Inputs: {inputs}")

# Fill form with output
form.fill(output={"analysis": "detailed analysis here"})
print(f"Filled: {form.filled}")
print(f"Output: {form.output}")

## 3. Report - Schema Registry Pattern

A `Report` is a namespace where class attributes define output schemas. This enables:
- Type introspection for validation
- Automatic request_model resolution
- Schema evolution tracking

In [ ]:
# Define output schemas
class Analysis(BaseModel):
    """Analysis of a topic."""

    summary: str = Field(description="Brief summary")
    key_points: list[str] = Field(description="Key points")
    challenges: list[str] = Field(description="Main challenges")


class Insights(BaseModel):
    """Insights derived from analysis."""

    patterns: list[str] = Field(description="Patterns observed")
    opportunities: list[str] = Field(description="Opportunities")
    risks: list[str] = Field(description="Potential risks")


class Recommendations(BaseModel):
    """Final recommendations."""

    actions: list[str] = Field(description="Recommended actions")
    priorities: list[str] = Field(description="Priority order")
    next_steps: str = Field(description="Immediate next steps")

In [ ]:
# Define Report with class attributes as schema registry
class TopicAnalysisReport(Report):
    """Workflow for analyzing a topic and producing recommendations."""

    # Output schemas (optional until filled)
    analysis: Analysis | None = None
    insights: Insights | None = None
    recommendations: Recommendations | None = None

    # Workflow definition
    assignment: str = "topic -> recommendations"
    form_assignments: list[str] = [
        "topic -> analysis",
        "analysis -> insights",
        "insights -> recommendations",
    ]


report = TopicAnalysisReport()
print(f"Input fields: {report.input_fields}")
print(f"Output fields: {report.output_fields}")
print(f"Number of forms: {len(list(report.forms))}")

In [ ]:
# Type introspection from class attributes
print(f"analysis type: {report.get_field_type('analysis')}")
print(f"insights type: {report.get_field_type('insights')}")
print(f"recommendations type: {report.get_field_type('recommendations')}")

# Pydantic models for structured output
print(f"\nrequest_model for 'analysis': {report.get_request_model('analysis')}")

In [ ]:
# Initialize with input data
report.initialize(topic="AI coding assistants in software development")

print(f"Available data: {report.available_data}")
print(f"Progress: {report.progress}")
print(f"Is complete: {report.is_complete()}")

In [ ]:
# Get next workable forms
next_forms = report.next_forms()
print(f"Workable forms: {next_forms}")
print(f"Assignment: {next_forms[0].assignment if next_forms else 'none'}")

## 4. Parallel Execution Pattern

Independent forms can run in parallel. The `flow_report` function builds a dependency graph and executes forms using `DependencyAwareExecutor`.

In [ ]:
# Diamond dependency pattern - a and b run in parallel, c waits for both
class DiamondReport(Report):
    a_result: str | None = None
    b_result: str | None = None
    c_result: str | None = None

    assignment: str = "input -> c_result"
    form_assignments: list[str] = [
        "input -> a_result",  # Parallel
        "input -> b_result",  # Parallel
        "a_result, b_result -> c_result",  # Waits for both
    ]


diamond = DiamondReport()
diamond.initialize(input="start")

# Both a and b are workable initially
next_forms = diamond.next_forms()
print(f"Initially workable: {[f.assignment for f in next_forms]}")

## 5. Full Workflow Execution

The `flow_report` function executes the entire workflow with automatic scheduling.

In [ ]:
# Setup session and model
model = iModel(
    provider="openai",
    model="gpt-4.1-mini",
    name="gpt4mini",
)
session = Session(default_generate_model=model)

# Create branch with capabilities
branch = session.create_branch(
    name="workflow",
    capabilities={"analysis", "insights", "recommendations"},
    resources={"gpt4mini"},
)

In [ ]:
# Create and initialize report
report = TopicAnalysisReport()
report.initialize(
    topic="The adoption of AI coding assistants in software development teams. "
    "Consider productivity impacts, code quality, learning curves, and team dynamics."
)

print(f"Workflow: {report.assignment}")
print(f"Forms: {report.form_assignments}")

In [ ]:
# Execute workflow
deliverable = await flow_report(
    session=session,
    branch=branch,
    report=report,
    verbose=True,
)

print(f"\nDeliverable keys: {list(deliverable.keys())}")

In [ ]:
# Inspect results
recommendations = deliverable.get("recommendations")
if recommendations and hasattr(recommendations, "model_dump"):
    rec_data = recommendations.model_dump()
    print("Recommendations:")
    print(f"  Actions: {rec_data.get('actions', [])}")
    print(f"  Priorities: {rec_data.get('priorities', [])}")
    print(f"  Next steps: {rec_data.get('next_steps', '')}")

In [ ]:
# View all completed forms
print("\nCompleted Forms:")
for form in report.completed_forms:
    print(f"  {form.assignment}: {type(form.output).__name__}")

## 6. Key Benefits

1. **Declarative**: Define workflows as data flow, not control flow
2. **Type-Safe**: Class attributes provide schema registry with introspection
3. **Parallel**: Independent forms execute concurrently automatically
4. **Composable**: Reports can be nested or combined
5. **Observable**: Progress tracking built-in via `progress` and `completed_forms`